In [1]:
%pip install rank-bm25
%pip install -U sentence-transformers
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 9.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import os
import unicodedata
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from typing import List, Dict, Tuple
import re
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import chromadb

2025-11-23 14:32:45.147133: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763908365.362669      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763908365.425471      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [4]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
TRAIN_FILE = "/kaggle/input/vlqa-dataset/train.json"
LEGAL_CORPUS_FILE = "/kaggle/input/vlqa-dataset/legal_corpus.json"
OUTPUT_FILE = "/kaggle/working/Data/hard_negative.json"

In [6]:
# Dense Retrieval Model 
embedding_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding", device="cuda")
embedding_model.half()

# Chunking settings
CHUNK_SIZE = 2048  
CHUNK_OVERLAP = 128

# Retrieval settings
BM25_TOP_K = 20  # Top 20 negatives từ BM25
DENSE_RERANK_TOP_K = 5  # Top 5 sau khi rerank bằng dense retrieval 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [7]:
# Load training data
with open(TRAIN_FILE, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

# Load legal corpus
with open(LEGAL_CORPUS_FILE, 'r', encoding='utf-8') as f:
    legal_corpus = json.load(f)

In [8]:
# mapping: article_id -> article_content
article_id_to_content = {}
article_id_to_text = {}
all_article_ids = []

for doc in tqdm(legal_corpus, desc="Procesisng legal corpus"):
    for article in doc.get('content', []):
        aid = article.get('aid')
        content = article.get('content_Article', '')
        if aid is not None and content:
            article_id_to_content[aid] = content
            article_id_to_text[aid] = content
            all_article_ids.append(aid)

Procesisng legal corpus: 100%|██████████| 2157/2157 [00:00<00:00, 30943.63it/s]


In [9]:
def chunk_with_overlap(document: str, chunk_size: int = 2000, overlap: int = 200) -> List[str]:
    """
    Chunk document into long chunks with overlap.
    Split at line breaks (\n) to maintain semantic continuity.
    
    Args:
        document: Input document text
        chunk_size: Maximum size of each chunk (characters)
        overlap: Number of overlapping characters between chunks
    
    Returns:
        List of text chunks
    """
    if not document:
        return []
    
    # Split document by line breaks first
    lines = document.split('\n')
    
    chunks = []
    current_chunk = ""
    
    for line in lines:
        # If adding this line would exceed chunk_size
        if len(current_chunk) + len(line) + 1 > chunk_size:
            if current_chunk:
                chunks.append(current_chunk.strip())
            
            # Start new chunk with overlap from previous chunk
            if chunks and overlap > 0:
                # Take last 'overlap' characters from previous chunk
                prev_chunk = chunks[-1]
                overlap_text = prev_chunk[-overlap:] if len(prev_chunk) > overlap else prev_chunk
                current_chunk = overlap_text + "\n" + line
            else:
                current_chunk = line
        else:
            if current_chunk:
                current_chunk += "\n" + line
            else:
                current_chunk = line
    
    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks
    
def preprocess(sentence: str) -> str:
    """Preprocess text: normalize, clean, remove special characters"""
    if not isinstance(sentence, str):
        return ""

    # Normalize Unicode
    sentence = unicodedata.normalize("NFC", sentence)

    # Lowercase
    sentence = sentence.lower().strip()

    # Remove URLs
    sentence = re.sub(r"https?://\S+|www\.\S+", "", sentence)

    # Remove email
    sentence = re.sub(r"\S+@\S+", "", sentence)

    # Remove special characters
    sentence = re.sub(r"[^a-zA-Z0-9À-ỹ ]+", " ", sentence)

    # Remove extra whitespace
    sentence = " ".join(sentence.split())

    return sentence

def tokenize(sentence: str) -> List[str]:

    if not sentence:
        return []

    # Preprocess
    sentence = preprocess(sentence)

    if not sentence:
        return []

    # NLTK word_tokenize
    result = word_tokenize(sentence)

    return result

# Test preprocessing & tokenization
test_text = "Thưa luật sư tôi có đăng; ký kết hôn (theo quy định tại Điều 1)"
print(f"Original: {test_text}")
print(f"Preprocessed: {preprocess(test_text)}")
print(f"Tokenized: {tokenize(test_text)}")

Original: Thưa luật sư tôi có đăng; ký kết hôn (theo quy định tại Điều 1)
Preprocessed: thưa luật sư tôi có đăng ký kết hôn theo quy định tại điều 1
Tokenized: ['thưa', 'luật', 'sư', 'tôi', 'có', 'đăng', 'ký', 'kết', 'hôn', 'theo', 'quy', 'định', 'tại', 'điều', '1']


In [10]:
# Chunking articles thành các đoạn dài (2048 chars với overlap)
chunked_data = []  # List of (chunk_id, article_id, chunk_text)
chunk_id_to_article_id = {}  # Mapping từ chunk_id -> article_id

chunk_id = 0
for aid in tqdm(all_article_ids, desc="Chunking articles"):
    content = article_id_to_text[aid]
    chunks = chunk_with_overlap(content, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP)
    
    for chunk_text in chunks:
        if chunk_text.strip():  
            chunked_data.append({
                'chunk_id': chunk_id,
                'article_id': aid,
                'chunk_text': chunk_text
            })
            chunk_id_to_article_id[chunk_id] = aid
            chunk_id += 1

tokenized_documents = []
article_ids_list = []

for aid in tqdm(all_article_ids, desc="Tokenizing articles"):
    content = article_id_to_text[aid]
    tokens = tokenize(content)
    tokenized_documents.append(tokens)
    article_ids_list.append(aid)

# Build BM25 index (Statistical Retrieval Model)
bm25 = BM25Okapi(tokenized_documents)

Tokenizing articles: 100%|██████████| 59635/59635 [00:59<00:00, 997.82it/s] 


In [11]:
# Embedding chunks và lưu vào ChromaDB
# Init ChromaDB 
chroma_client = chromadb.PersistentClient(path="/kaggle/working/chroma_db")
collection_name = "legal_articles_chunks"

# Load hoặc tạo collection
try:
    collection = chroma_client.get_collection(name=collection_name)
    collection_count = collection.count()
    need_to_add = collection_count < len(chunked_data)
    print(f"Collection đã tồn tại với {collection_count} chunks")
except:
    collection = chroma_client.create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"}
    )
    need_to_add = True
    print("Tạo collection mới")

# Nếu cần thì embed và add theo batch nhỏ để tránh lỗi
if need_to_add:
    chunk_texts = [item['chunk_text'] for item in chunked_data]
    chunk_ids   = [str(item['chunk_id']) for item in chunked_data]
    metadatas   = [
        {
            'article_id': str(item['article_id']),
            'chunk_text': item['chunk_text'][:500]
        }
        for item in chunked_data
    ]

    # Tạo embeddings theo batch
    embedding_batch_size = 8
    db_batch_size = 5000  # Batch size nhỏ hơn để tránh lỗi ChromaDB
    
    all_embeddings = []
    for i in tqdm(range(0, len(chunk_texts), embedding_batch_size), desc="Embedding chunks"):
        batch = chunk_texts[i:i+embedding_batch_size]
        vecs = embedding_model.encode(batch, show_progress_bar=False)
        all_embeddings.extend(vecs.tolist())

    # Add vào ChromaDB theo batch nhỏ
    for i in tqdm(range(0, len(chunk_ids), db_batch_size), desc="Adding to ChromaDB"):
        batch_ids = chunk_ids[i:i+db_batch_size]
        batch_embeddings = all_embeddings[i:i+db_batch_size]
        batch_documents = chunk_texts[i:i+db_batch_size]
        batch_metadatas = metadatas[i:i+db_batch_size]
        
        collection.add(
            ids=batch_ids,
            embeddings=batch_embeddings,
            documents=batch_documents,
            metadatas=batch_metadatas
        )
    
    print(f"✅ Đã thêm {len(chunk_ids)} chunks vào ChromaDB")

# Load mapping chunk_id -> article_id từ ChromaDB
existing = collection.get()
for cid, meta in zip(existing['ids'], existing['metadatas']):
    if meta.get('article_id') is not None:
        chunk_id_to_article_id[int(cid)] = int(meta['article_id'])

print(f"✅ Đã tải {len(chunk_id_to_article_id)} mappings từ ChromaDB")

Tạo collection mới


Adding to ChromaDB: 100%|██████████| 16/16 [02:35<00:00,  9.74s/it]


✅ Đã thêm 78229 chunks vào ChromaDB
✅ Đã tải 78229 mappings từ ChromaDB


In [12]:
# Hàm retrieve top-k articles từ ChromaDB (không dùng RAM)
def retrieve_top_k_from_chromadb(question: str, top_k: int = 100) -> List[Tuple[int, float]]:
    """
    Retrieve top-k chunks từ ChromaDB sử dụng dense retrieval.
    Không load embeddings vào RAM, query trực tiếp từ ChromaDB.
    
    Returns:
        List of tuples (article_id, score) sorted by score descending
    """
    # Encode question
    query_embedding = embedding_model.encode([question], show_progress_bar=False)[0]
    
    # Query ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=top_k
    )
    
    # Extract article_ids và scores
    retrieved_articles = []
    if results['ids'] and len(results['ids'][0]) > 0:
        for i, chunk_id in enumerate(results['ids'][0]):
            article_id = chunk_id_to_article_id.get(int(chunk_id))
            if article_id is not None:
                # Lấy distance từ ChromaDB (cosine distance)
                distance = results['distances'][0][i] if 'distances' in results else 0.0
                # Convert distance to similarity score (1 - distance for cosine)
                score = 1.0 - distance
                retrieved_articles.append((article_id, score))
    
    # Remove duplicates và sort by score
    article_scores = {}
    for aid, score in retrieved_articles:
        if aid not in article_scores or score > article_scores[aid]:
            article_scores[aid] = score
    
    results_sorted = sorted(article_scores.items(), key=lambda x: x[1], reverse=True)
    return results_sorted[:top_k]

# Hàm retrieve top-k articles với BM25
def retrieve_top_k_bm25(question: str, top_k: int = 100) -> List[Tuple[int, float]]:
    """
    Retrieve top-k articles sử dụng BM25.
    
    Returns:
        List of tuples (article_id, bm25_score) sorted by score descending
    """
    query_tokens = tokenize(question)
    bm25_scores = bm25.get_scores(query_tokens)
    
    # Tạo list (article_id, bm25_score) và sort
    results = [(article_ids_list[i], float(bm25_scores[i])) for i in range(len(bm25_scores))]
    results.sort(key=lambda x: x[1], reverse=True)
    
    return results[:top_k]

# Test retrieval
test_question = train_data[0]['question']
print(f"\nTest question: {test_question[:100]}...")

# Test BM25
bm25_results = retrieve_top_k_bm25(test_question, top_k=10)
print(f"\nTop 10 retrieved articles (BM25):")
for aid, score in bm25_results[:10]:
    print(f"  Article ID: {aid}, BM25 Score: {score:.4f}")

# Test ChromaDB
chromadb_results = retrieve_top_k_from_chromadb(test_question, top_k=10)
print(f"\nTop 10 retrieved articles (ChromaDB Dense Retrieval):")
for aid, score in chromadb_results[:10]:
    print(f"  Article ID: {aid}, Similarity Score: {score:.4f}")


Test question: Thưa luật sư tôi có đăng ký kết hôn trên pháp luật nhưng nay vợ chồng bỏ nhau theo phong tục tập quá...

Top 10 retrieved articles (BM25):
  Article ID: 3441, BM25 Score: 85.9256
  Article ID: 53886, BM25 Score: 85.0300
  Article ID: 6740, BM25 Score: 83.5712
  Article ID: 3424, BM25 Score: 83.2882
  Article ID: 5850, BM25 Score: 82.1975
  Article ID: 53875, BM25 Score: 81.8095
  Article ID: 5856, BM25 Score: 80.5794
  Article ID: 5849, BM25 Score: 79.6637
  Article ID: 53939, BM25 Score: 79.5563
  Article ID: 53877, BM25 Score: 75.5946

Top 10 retrieved articles (ChromaDB Dense Retrieval):
  Article ID: 57800, Similarity Score: 0.5199
  Article ID: 53875, Similarity Score: 0.4965
  Article ID: 53877, Similarity Score: 0.4689
  Article ID: 53881, Similarity Score: 0.4683
  Article ID: 53886, Similarity Score: 0.4535
  Article ID: 53880, Similarity Score: 0.4506
  Article ID: 53882, Similarity Score: 0.4468
  Article ID: 5168, Similarity Score: 0.4380
  Article ID: 53894

In [14]:
# BATCH PROCESSING VERSION - Tối ưu hơn
# Tạo hard negative data với batch processing để tăng tốc độ

print("\nĐang tạo hard negative data với BATCH PROCESSING...")

# Phase 1: BM25 retrieval cho tất cả questions
print("Phase 1: BM25 retrieval...")
all_questions_data = []
for item in tqdm(train_data, desc="BM25 retrieval"):
    qid = item['qid']
    question = item['question']
    relevant_laws = set(item['relevant_laws'])
    
    bm25_results = retrieve_top_k_bm25(question, top_k=100)
    bm25_negatives = []
    for aid, bm25_score in bm25_results:
        if aid not in relevant_laws:
            bm25_negatives.append({
                'article_id': aid,
                'article_content': article_id_to_content.get(aid, ''),
                'bm25_score': float(bm25_score)
            })
        if len(bm25_negatives) >= BM25_TOP_K:
            break
    
    all_questions_data.append({
        'qid': qid,
        'question': question,
        'relevant_laws': relevant_laws,
        'bm25_negatives': bm25_negatives
    })


Đang tạo hard negative data với BATCH PROCESSING...
Phase 1: BM25 retrieval...


BM25 retrieval: 100%|██████████| 2190/2190 [18:44<00:00,  1.95it/s]



Phase 2: Batch encoding và reranking...


Processing batches:   3%|▎         | 2/69 [02:00<1:07:27, 60.41s/it]


KeyboardInterrupt: 

In [18]:
# Phase 2: Batch encoding và reranking (TẬN DỤNG CHROMADB)
print("\nPhase 2: Batch encoding và reranking (sử dụng ChromaDB query)...")
hard_negative_data = []

# Batch encode questions một lần
print("  - Encoding questions (batch)...")
all_questions = [q_data['question'] for q_data in all_questions_data]
question_embeddings_list = embedding_model.encode(all_questions, show_progress_bar=True, batch_size=32)
question_embeddings_dict = {q_data['qid']: emb for q_data, emb in zip(all_questions_data, question_embeddings_list)}

# Cache để lưu article embeddings từ ChromaDB queries (tránh tính lại)
article_embeddings_cache = {}  # article_id -> embedding (average của chunks)

print("  - Reranking negatives (query ChromaDB cho mỗi question)...")
for q_idx, q_data in tqdm(enumerate(all_questions_data), desc="Processing questions", total=len(all_questions_data)):
    qid = q_data['qid']
    question = q_data['question']
    relevant_laws = q_data['relevant_laws']
    bm25_negatives = q_data['bm25_negatives']
    
    # Lấy question embedding
    question_emb = question_embeddings_dict[qid]
    
    # Query ChromaDB để lấy top chunks (tận dụng embeddings đã có)
    query_embedding = question_emb.tolist()
    chroma_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=min(500, collection.count()),  # Query nhiều chunks để có đủ articles
        include=["embeddings", "metadatas"]  # Lấy embeddings và metadatas
    )
    
    # Extract chunk embeddings và article_ids từ ChromaDB results
    chunk_ids = chroma_results['ids'][0] if chroma_results.get('ids') and len(chroma_results['ids']) > 0 else []
    chunk_embeddings = chroma_results['embeddings'][0] if chroma_results.get('embeddings') and len(chroma_results['embeddings']) > 0 else []
    
    # Tạo mapping article_id -> list of chunk embeddings
    article_to_chunks = {}
    for chunk_id, chunk_emb in zip(chunk_ids, chunk_embeddings):
        article_id = chunk_id_to_article_id.get(int(chunk_id))
        if article_id is not None:
            if article_id not in article_to_chunks:
                article_to_chunks[article_id] = []
            article_to_chunks[article_id].append(chunk_emb)
    
    # Average pool để có article embeddings và cache
    article_embeddings_from_chroma = {}
    for article_id, chunk_embs in article_to_chunks.items():
        article_emb = np.mean(np.array(chunk_embs), axis=0)
        article_embeddings_from_chroma[article_id] = article_emb
        # Cache để dùng lại
        if article_id not in article_embeddings_cache:
            article_embeddings_cache[article_id] = article_emb
    
    # Rerank negatives: lấy article embeddings từ ChromaDB query results
    article_embeddings_list = []
    articles_to_encode = []  # Articles không có trong ChromaDB results
    
    for neg in bm25_negatives:
        article_id = neg['article_id']
        
        # Thử lấy từ ChromaDB query results trước
        if article_id in article_embeddings_from_chroma:
            article_embeddings_list.append(article_embeddings_from_chroma[article_id])
        elif article_id in article_embeddings_cache:
            article_embeddings_list.append(article_embeddings_cache[article_id])
        else:
            # Nếu không có, sẽ encode sau
            articles_to_encode.append((len(article_embeddings_list), neg))
            article_embeddings_list.append(None)  # Placeholder
    
    # Encode các articles không có trong ChromaDB
    if articles_to_encode:
        texts_to_encode = [neg['article_content'] for _, neg in articles_to_encode]
        encoded_embs = embedding_model.encode(texts_to_encode, show_progress_bar=False, batch_size=16)
        
        for (idx, _), emb in zip(articles_to_encode, encoded_embs):
            article_embeddings_list[idx] = emb
    
    # Tính similarity
    if len(article_embeddings_list) > 0:
        # Lọc bỏ None và tạo mapping
        valid_embeddings = []
        valid_indices = []
        
        for i, emb in enumerate(article_embeddings_list):
            if emb is not None:
                valid_embeddings.append(emb)
                valid_indices.append(i)
        
        if len(valid_embeddings) > 0:
            question_emb_reshaped = question_emb.reshape(1, -1)
            article_embeddings = np.array(valid_embeddings)
            similarities = cosine_similarity(question_emb_reshaped, article_embeddings)[0]
            
            # Tạo dict mapping index -> similarity
            similarity_dict = {valid_indices[i]: float(sim) for i, sim in enumerate(similarities)}
            
            rerank_results = [
                {
                    'article_id': neg['article_id'],
                    'article_content': neg['article_content'],
                    'bm25_score': neg['bm25_score'],
                    'dense_score': similarity_dict.get(i, 0.0)
                }
                for i, neg in enumerate(bm25_negatives)
                if i in similarity_dict
            ]
            rerank_results.sort(key=lambda x: x['dense_score'], reverse=True)
            hard_negatives = rerank_results[:DENSE_RERANK_TOP_K]
        else:
            hard_negatives = []
    else:
        hard_negatives = []
    
    # Tạo positive examples (luôn chạy, không phụ thuộc vào hard_negatives)
    positive_examples = []
    for aid in relevant_laws:
        if aid in article_id_to_content:
            positive_examples.append({
                'article_id': aid,
                'article_content': article_id_to_content[aid]
            })
    
    # Tạo training pairs
    for pos_article in positive_examples:
        hard_negative_data.append({
            'qid': qid,
            'question': question,
            'article_id': pos_article['article_id'],
            'article_content': pos_article['article_content'],
            'label': 1
        })
    
    for neg_article in hard_negatives:
        hard_negative_data.append({
            'qid': qid,
            'question': question,
            'article_id': neg_article['article_id'],
            'article_content': neg_article['article_content'],
            'bm25_score': neg_article['bm25_score'],
            'dense_score': neg_article['dense_score'],
            'label': 0
        })

print(f"\n✅ Đã tạo {len(hard_negative_data)} training pairs với batch processing")
print(f"  - Positive pairs: {sum(1 for x in hard_negative_data if x['label'] == 1)}")
print(f"  - Negative pairs: {sum(1 for x in hard_negative_data if x['label'] == 0)}")
print(f"  - ChromaDB cache: {len(article_embeddings_cache)} unique articles (tận dụng embeddings đã có)")
print(f"  - Questions encoded: {len(question_embeddings_dict)} (batch encoding)")


Phase 2: Batch encoding và reranking (sử dụng ChromaDB query)...
  - Encoding questions (batch)...


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

  - Reranking negatives (query ChromaDB cho mỗi question)...


Processing questions: 100%|██████████| 2190/2190 [04:55<00:00,  7.41it/s]


✅ Đã tạo 13886 training pairs với batch processing
  - Positive pairs: 2936
  - Negative pairs: 10950
  - ChromaDB cache: 57022 unique articles (tận dụng embeddings đã có)
  - Questions encoded: 2190 (batch encoding)


In [19]:
# Hiển thị ví dụ về data đã tạo
print("\n" + "="*80)
print("VÍ DỤ POSITIVE PAIR (Question + Relevant Article)")
print("="*80)
pos_example = [x for x in hard_negative_data if x['label'] == 1][0]
print(f"QID: {pos_example['qid']}")
print(f"Question: {pos_example['question']}")
print(f"Article ID: {pos_example['article_id']}")
print(f"Article Content: {pos_example['article_content'][:200]}...")
print(f"Label: {pos_example['label']} (Positive)")

print("\n" + "="*80)
print("VÍ DỤ NEGATIVE PAIR (Question + Hard Negative Article)")
print("="*80)
neg_example = [x for x in hard_negative_data if x['label'] == 0][0]
print(f"QID: {neg_example['qid']}")
print(f"Question: {neg_example['question']}")
print(f"Article ID: {neg_example['article_id']}")
print(f"Article Content: {neg_example['article_content'][:200]}...")
print(f"BM25 Score: {neg_example.get('bm25_score', 'N/A'):.4f}")
print(f"Dense Retrieval Score (Vietnamese_Embedding): {neg_example.get('dense_score', 'N/A'):.4f}")
print(f"Label: {neg_example['label']} (Negative - Hard Negative)")


VÍ DỤ POSITIVE PAIR (Question + Relevant Article)
QID: 933
Question: Thưa luật sư tôi có đăng ký kết hôn trên pháp luật nhưng nay vợ chồng bỏ nhau theo phong tục tập quán như vậy tôi có được phép kết hôn với người khác không ạ?
Article ID: 53929
Article Content: 1. Quan hệ hôn nhân chấm dứt kể từ ngày bản án, quyết định ly hôn của Tòa án có hiệu lực pháp luật.

2. Tòa án đã giải quyết ly hôn phải gửi bản án, quyết định ly hôn đã có hiệu lực pháp luật cho cơ q...
Label: 1 (Positive)

VÍ DỤ NEGATIVE PAIR (Question + Hard Negative Article)
QID: 933
Question: Thưa luật sư tôi có đăng ký kết hôn trên pháp luật nhưng nay vợ chồng bỏ nhau theo phong tục tập quán như vậy tôi có được phép kết hôn với người khác không ạ?
Article ID: 53881
Article Content: 1. Việc kết hôn phải được đăng ký và do cơ quan nhà nước có thẩm quyền thực hiện theo quy định của Luật này và pháp luật về hộ tịch.Việc kết hôn không được đăng ký theo quy định tại khoản này thì khôn...
BM25 Score: 71.3940
Dense Retrieval Sco

In [24]:
# Tạo training data cho BGE-reranker-v2 finetuning
# Format: JSONL với mỗi dòng là {"query": "...", "passage": "...", "label": 1/0}

print("\n" + "="*80)
print("TẠO TRAINING DATA CHO BGE-RERANKER-V2 FINETUNING")
print("="*80)

# Format 1: JSONL format (mỗi dòng 1 sample)
# Format: {"query": "...", "passage": "...", "label": 1/0}
BGE_TRAIN_FILE_JSONL = "/kaggle/working/bge_reranker_train.json"

print(f"\nTạo JSONL format cho BGE-reranker-v2...")
with open(BGE_TRAIN_FILE_JSONL, 'w', encoding='utf-8') as f:
    for item in tqdm(hard_negative_data, desc="Writing JSONL"):
        sample = {
            "query": item['question'],
            "passage": item['article_content'],
            "label": item['label']
        }
        f.write(json.dumps(sample, ensure_ascii=False) + '\n')

print(f"✅ Đã tạo {len(hard_negative_data)} samples trong {BGE_TRAIN_FILE_JSONL}")

# Format 2: Pair format (cho FlagEmbedding training)
# Format: List of [query, passage] pairs với labels
BGE_TRAIN_FILE_PAIRS = "/kaggle/working/bge_reranker_train_pairs.json"

print(f"\nTạo Pair format cho FlagEmbedding training...")
bge_pairs_data = []
for item in hard_negative_data:
    bge_pairs_data.append({
        "query": item['question'],
        "passage": item['article_content'],
        "label": item['label']
    })

with open(BGE_TRAIN_FILE_PAIRS, 'w', encoding='utf-8') as f:
    json.dump(bge_pairs_data, f, ensure_ascii=False, indent=2)

print(f"✅ Đã tạo {len(bge_pairs_data)} pairs trong {BGE_TRAIN_FILE_PAIRS}")

# Format 3: Triplet format (query, positive, negative) - cho contrastive learning
# Tạo triplets: mỗi question có 1 positive và 1 negative
BGE_TRAIN_FILE_TRIPLETS = "/kaggle/working/bge_reranker_train_triplets.json"

print(f"\nTạo Triplet format (query, positive, negative)...")
triplets_data = []

# Group by qid
qid_to_samples = {}
for item in hard_negative_data:
    qid = item['qid']
    if qid not in qid_to_samples:
        qid_to_samples[qid] = {
            'question': item['question'],  # Lưu question luôn
            'positives': [], 
            'negatives': []
        }
    
    if item['label'] == 1:
        qid_to_samples[qid]['positives'].append(item['article_content'])
    else:
        qid_to_samples[qid]['negatives'].append(item['article_content'])

# Tạo triplets: mỗi positive pair với 1 negative
for qid, samples in tqdm(qid_to_samples.items(), desc="Creating triplets"):
    question = samples['question']
    positives = samples['positives']
    negatives = samples['negatives']
    
    # Tạo triplets: mỗi positive với 1 negative
    # Nếu có nhiều negatives, có thể tạo nhiều triplets
    for pos in positives:
        for neg in negatives[:min(5, len(negatives))]:  # Tối đa 3 negatives per positive
            triplets_data.append({
                "query": question,
                "positive": pos,
                "negative": neg
            })

with open(BGE_TRAIN_FILE_TRIPLETS, 'w', encoding='utf-8') as f:
    json.dump(triplets_data, f, ensure_ascii=False, indent=2)

print(f"✅ Đã tạo {len(triplets_data)} triplets trong {BGE_TRAIN_FILE_TRIPLETS}")

# Thống kê
print(f"\n" + "="*80)
print("THỐNG KÊ TRAINING DATA CHO BGE-RERANKER-V2")
print("="*80)
print(f"  - Tổng số samples: {len(hard_negative_data)}")
print(f"  - Positive samples: {sum(1 for x in hard_negative_data if x['label'] == 1)}")
print(f"  - Negative samples: {sum(1 for x in hard_negative_data if x['label'] == 0)}")
print(f"  - Tổng số triplets: {len(triplets_data)}")
print(f"\n  - JSONL format: {BGE_TRAIN_FILE_JSONL}")
print(f"  - Pairs format: {BGE_TRAIN_FILE_PAIRS}")
print(f"  - Triplets format: {BGE_TRAIN_FILE_TRIPLETS}")
print("\n✅ Hoàn thành tạo training data cho BGE-reranker-v2!")


TẠO TRAINING DATA CHO BGE-RERANKER-V2 FINETUNING

Tạo JSONL format cho BGE-reranker-v2...


Writing JSONL: 100%|██████████| 13886/13886 [00:00<00:00, 19894.15it/s]


✅ Đã tạo 13886 samples trong /kaggle/working/bge_reranker_train.json

Tạo Pair format cho FlagEmbedding training...
✅ Đã tạo 13886 pairs trong /kaggle/working/bge_reranker_train_pairs.json

Tạo Triplet format (query, positive, negative)...


Creating triplets: 100%|██████████| 2190/2190 [00:00<00:00, 293411.03it/s]


✅ Đã tạo 14680 triplets trong /kaggle/working/bge_reranker_train_triplets.json

THỐNG KÊ TRAINING DATA CHO BGE-RERANKER-V2
  - Tổng số samples: 13886
  - Positive samples: 2936
  - Negative samples: 10950
  - Tổng số triplets: 14680

  - JSONL format: /kaggle/working/bge_reranker_train.json
  - Pairs format: /kaggle/working/bge_reranker_train_pairs.json
  - Triplets format: /kaggle/working/bge_reranker_train_triplets.json

✅ Hoàn thành tạo training data cho BGE-reranker-v2!


In [23]:
# Hiển thị ví dụ về các format đã tạo
print("\n" + "="*80)
print("VÍ DỤ CÁC FORMAT TRAINING DATA")
print("="*80)

# Ví dụ JSONL format
print("\n1. JSONL FORMAT (bge_reranker_train.jsonl):")
print("-" * 80)
with open(BGE_TRAIN_FILE_JSONL, 'r', encoding='utf-8') as f:
    first_line = f.readline()
    sample = json.loads(first_line)
    print(json.dumps(sample, ensure_ascii=False, indent=2))
    print(f"\n  (Mỗi dòng là 1 JSON object)")

# Ví dụ Pairs format
print("\n2. PAIRS FORMAT (bge_reranker_train_pairs.json):")
print("-" * 80)
with open(BGE_TRAIN_FILE_PAIRS, 'r', encoding='utf-8') as f:
    pairs_data = json.load(f)
    print(json.dumps(pairs_data[:2], ensure_ascii=False, indent=2))
    print(f"\n  (List of objects với query, passage, label)")

# Ví dụ Triplets format
print("\n3. TRIPLETS FORMAT (bge_reranker_train_triplets.json):")
print("-" * 80)
with open(BGE_TRAIN_FILE_TRIPLETS, 'r', encoding='utf-8') as f:
    triplets_data = json.load(f)
    if len(triplets_data) > 0:
        print(json.dumps(triplets_data[0], ensure_ascii=False, indent=2))
        print(f"\n  (Query với 1 positive và 1 negative passage)")


VÍ DỤ CÁC FORMAT TRAINING DATA

1. JSONL FORMAT (bge_reranker_train.jsonl):
--------------------------------------------------------------------------------
{
  "query": "Thưa luật sư tôi có đăng ký kết hôn trên pháp luật nhưng nay vợ chồng bỏ nhau theo phong tục tập quán như vậy tôi có được phép kết hôn với người khác không ạ?",
  "passage": "1. Quan hệ hôn nhân chấm dứt kể từ ngày bản án, quyết định ly hôn của Tòa án có hiệu lực pháp luật.\n\n2. Tòa án đã giải quyết ly hôn phải gửi bản án, quyết định ly hôn đã có hiệu lực pháp luật cho cơ quan đã thực hiện việc đăng ký kết hôn để ghi vào sổ hộ tịch; hai bên ly hôn; cá nhân, cơ quan, tổ chức khác theo quy định của Bộ luật tố tụng dân sự và các luật khác có liên quan.",
  "label": 1
}

  (Mỗi dòng là 1 JSON object)

2. PAIRS FORMAT (bge_reranker_train_pairs.json):
--------------------------------------------------------------------------------
[
  {
    "query": "Thưa luật sư tôi có đăng ký kết hôn trên pháp luật nhưng nay vợ chồng bỏ

In [25]:
!zip -r chroma_db.zip /kaggle/working/chroma_db

  adding: kaggle/working/chroma_db/ (stored 0%)
  adding: kaggle/working/chroma_db/chroma.sqlite3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 60%)
  adding: kaggle/working/chroma_db/fcf5f382-6f3c-4073-91f7-cff8045f3413/ (stored 0%)
  adding: kaggle/working/chroma_db/fcf5f382-6f3c-4073-91f7-cff8045f3413/header.bin (deflated 54%)
  adding: kaggle/working/chroma_db/fcf5f382-6f3c-4073-91f7-cff8045f3413/link_lists.bin (deflated 68%)
  adding: kaggle/working/chroma_db/fcf5f382-6f3c-4073-91f7-cff8045f3413/length.bin (deflated 44%)
  adding: kaggle/working/chroma_db/fcf5f382-6f3c-4073-91f7-cff8045f3413/data_level0.bin (deflated 16%)
  adding: kaggle/working/chroma_db/fcf5f382-6f3c-4073-91f7-cff8045f3413/index_metadata.pickle (deflated 60%)
